In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 33170850
paper_name = 'yang_tavazoie_2020' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [15]:
original_data = pd.read_csv('raw_data/Yang_fitness_scores_sorted.txt', header=None, names=['orf','data'], sep='\t')

In [16]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 3804 x 2


In [17]:
original_data.head()

,orf,data
0,YPR087W,-12.3952570401636
1,YNL032W,-6.92104200538018
2,YDR176W,-6.81150323562367
3,YJL047C-A,-6.78707129464744
4,YOR245C,-6.38384858883437


In [18]:
original_data['orf'] = original_data['orf'].apply(lambda x: x.split('_')[0])

In [19]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [20]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [21]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

              orf            data
index_input                      
2074         GENE  log2FoldChange


In [22]:
original_data = original_data.loc[t,]

In [23]:
original_data['data'] = pd.to_numeric(original_data['data'], errors='coerce')

In [24]:
original_data.set_index('orf', inplace=True)

In [25]:
original_data = original_data[['data']].copy()

In [26]:
original_data = original_data.groupby(original_data.index).mean()

In [27]:
original_data.shape

(3754, 1)

# Prepare the final dataset

In [28]:
data = original_data.copy()

In [29]:
dataset_ids = [16708]
datasets = datasets.reindex(index=dataset_ids)

In [30]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [31]:
data.head()

dataset_id,16708
data_type,value
orf,
YAL012W,0.803232
YAL021C,-0.639495
YAL024C,-0.055627
YAL044W-A,-0.233053
YAL054C,0.192453


## Subset to the genes currently in SGD

In [32]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 18


In [33]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16708
,data_type,value
gene_id,orf,
10,YAL012W,0.803232
19,YAL021C,-0.639495
22,YAL024C,-0.055627
6180,YAL044W-A,-0.233053
50,YAL054C,0.192453


# Normalize

In [34]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [35]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [36]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id            16708          
data_type             value    valuez
gene_id orf                          
10      YAL012W    0.803232  0.437398
19      YAL021C   -0.639495 -0.490002
22      YAL024C   -0.055627 -0.114685
6180    YAL044W-A -0.233053 -0.228737
50      YAL054C    0.192453  0.044783

# Print out

In [37]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [39]:
from IO.save_data_to_db3 import *

In [40]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 33170850...
Inserting the new data...


100%|██████████| 1/1 [00:06<00:00,  6.08s/it]

Updating the data_modified_on field...
